## Gensim - LDA

In [6]:
import warnings
warnings.filterwarnings("ignore") 

import pandas as pd
import numpy as np
import pickle
#import helper as h

import re
import string
import gensim
from gensim import corpora, models
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import nltk
#nltk.download('wordnet')

pyLDAvis.enable_notebook()
%config Completer.use_jedi = False

def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [7]:
#!pip install gensim
#!pip install pyLDAvis
# nltk.download('wordnet')

In [9]:
#get all data
df_articles = pd.read_csv("../dataset/News_articles_dataset/News_articles_dataset.csv.gz")
df_articles.sample(5)

,media,date,url,heading,content,other.author,other.top_image,other.category,city,year,state,district,matches
11932,Hindu,2011-06-03,https://www.thehindu.com/features/metroplus/ba...,Back to a simple life,Stymie the excesses of consumerism by a return...,V.R.MENON,https://www.thehindu.com/static/theme/default/...,metroplus,NaN,NaN,NaN,NaN,{'ozone': 1.0}
5410,TOI,2017-10-26,http://timesofindia.indiatimes.com//city/noida...,Construction work and snarls never stop here,NOIDA: Sector 125 in Noida is quite the opposi...,"TNN / Oct 26, 2017, 03:01 IST","https://static.toiimg.com/thumb/msid-61232385,...",NaN,noida,2017.0,uttar pradesh,gautam buddha nagar,"{'air quality': 2.0, 'aqi': 1.0}"
10213,TOI,2020-03-23,http://timesofindia.indiatimes.com//india/jant...,Janta curfew: Empty roads & the sound of silence,Janta curfew: Empty roads & the sound of silen...,"TNN / Mar 23, 2020, 04:47 IST",NaN,NewsIndia NewsJanta curfew: Empty roads & the ...,NaN,2020.0,NaN,NaN,{'air quality': 1.0}
2757,TOI,2015-10-29,http://timesofindia.indiatimes.com//city/delhi...,"Needed, air quality curbs on firecrackers",NEW DELHI: Diwali is just over a fortnight awa...,"TNN / Updated: Oct 29, 2015, 04:18 IST","https://static.toiimg.com/thumb/msid-47529300,...",NaN,delhi,2015.0,delhi,central delhi,"{'air pollution': 3.0, 'air quality': 3.0}"
3702,TOI,2016-05-10,http://timesofindia.indiatimes.com//city/delhi...,Why PM emission from fly ash can't be ignored,New Delhi: Natural dust may be one of the bigg...,"Jayashree Nandi / TNN / May 10, 2016, 00:30 IST","https://static.toiimg.com/thumb/msid-47529300,...",NaN,delhi,2016.0,delhi,central delhi,"{'air pollution': 1.0, 'air quality': 1.0, 'pm..."


In [10]:
def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = str(text)
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text)
    text = re.sub('â€™', '', text)
    text = re.sub('œ', '', text)
    re.sub(r'[^\x00-\x7F]+','', text)
    return text

def clean_text_round2(text):
    '''Get rid of some additional punctuation and non-sensical text that was missed the first time around.'''
    text = str(text)
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    return text

#not doing stemming.
def lemmatize_stemming(text):
    text = str(text)
    #stemmer = SnowballStemmer("english")
    return WordNetLemmatizer().lemmatize(text, pos='v')

def remove_nonascii(text):
    text = str(text)
    return ''.join([i if ord(i) < 128 else ' ' for i in text])

def preprocess(text):
    text = str(text)
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return " ".join(str(x) for x in result)

def remove_mention_hashtag(text):
    text = str(text)
    text = re.sub('# [a-zA-Z]*', '', text)
    text = re.sub('# [a-zA-Z]*', '', text)
    return text
    
def some_words(text):
    text = str(text)
    text = re.sub('air pollution', '', text)
    text = re.sub('pollution', '', text)
    return text

In [11]:
df = df_articles.copy()
df["heading_and_content"] = df.apply(lambda x:x["heading"]+" "+x["content"],axis=1)
df

,media,date,url,heading,content,other.author,other.top_image,other.category,city,year,state,district,matches,heading_and_content
0,TOI,2010-01-01,http://timesofindia.indiatimes.com//city/mumba...,Mumbai celebrates a cool New Year's Eve at 13....,MUMBAI: The city witnessed its lowest temperat...,"TNN / Jan 1, 2010, 01:47 IST","https://static.toiimg.com/thumb/msid-47529300,...",NaN,mumbai,2010.0,maharashtra,mumbai suburban,{'smog': 3.0},Mumbai celebrates a cool New Year's Eve at 13....
1,TOI,2010-01-02,http://timesofindia.indiatimes.com//city/luckn...,Dense fog leads to tripping of power lines,LUCKNOW: The UP Power Corporation Limited is k...,"TNN / Jan 2, 2010, 06:40 IST","https://static.toiimg.com/thumb/msid-47529300,...",NaN,lucknow,2010.0,uttar pradesh,lucknow,{'air pollutants': 2.0},Dense fog leads to tripping of power lines LUC...
2,TOI,2010-01-09,http://timesofindia.indiatimes.com//city/chenn...,City has high levels of smog-forming ozone,CHENNAI: Dangerous levels of ground-level ozon...,"Bhama Devi Ravi / TNN / Jan 9, 2010, 00:37 IST","https://static.toiimg.com/thumb/msid-47529300,...",NaN,chennai,2010.0,tamil nadu,chennai,"{'ozone': 9.0, 'smog': 4.0}",City has high levels of smog-forming ozone CHE...
3,TOI,2010-01-11,http://timesofindia.indiatimes.com//city/jaipu...,"Woman, four kids in hospital after inhaling smoke",JAIPUR: A woman and her four kids were admitte...,"TNN / Jan 11, 2010, 01:56 IST","https://static.toiimg.com/thumb/msid-47529300,...",NaN,jaipur,2010.0,rajasthan,jaipur,{'carbon monoxide': 1.0},"Woman, four kids in hospital after inhaling sm..."
4,TOI,2010-01-13,http://timesofindia.indiatimes.com//city/chenn...,Pollution high on Bhogi eve: Study,"CHENNAI: By the time you get to read this, the...","Bhama Devi Ravi / TNN / Jan 13, 2010, 04:23 IST","https://static.toiimg.com/thumb/msid-47529300,...",NaN,chennai,2010.0,tamil nadu,chennai,"{'air quality': 1.0, 'carbon monoxide': 1.0, '...",Pollution high on Bhogi eve: Study CHENNAI: By...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17369,Hindu,2021-05-16,https://www.thehindu.com/news/national/andhra-...,Naval Dockyard teams repair two oxygen plants ...,Oxygen supply gets a big boost with this in th...,SpecialCorrespondent,https://www.thehindu.com/news/national/andhra-...,andhra pradesh,visakhapatnam,NaN,andhra pradesh,visakhapatnam,{'carbon monoxide': 2.0},Naval Dockyard teams repair two oxygen plants ...
17370,Hindu,2021-05-24,https://www.thehindu.com/news/national/kerala/...,Greens call for a relook into SilverLine imple...,"The Kerala Paristhithi Aikya Vedi, a forum of ...",SpecialCorrespondent,https://www.thehindu.com/static/theme/default/...,kerala,thiruvananthapuram,NaN,kerala,thiruvananthapuram,{'air pollution': 1.0},Greens call for a relook into SilverLine imple...
17371,Hindu,2021-05-25,https://www.thehindu.com/news/national/karnata...,CFTRI’s ozone-based air disinfection system to...,The CSIR-Central Food Technological Research I...,LaiqhAKhan,https://www.thehindu.com/sci-tech/science/wsmd...,karnataka,mysuru,NaN,karnataka,mysuru,{'ozone': 9.0},CFTRI’s ozone-based air disinfection system to...
17372,Hindu,2021-05-28,https://www.thehindu.com/news/national/ngt-sta...,NGT stays green nod for NTPC’s Karimnagar plant,Environmental Clearance put on hold for seven ...,T.K.Rohit,https://www.thehindu.com/news/cities/Hyderabad...,national,chennai,NaN,tamil nadu,chennai,{'air quality': 1.0},NGT stays green nod for NTPC’s Karimnagar plan...


In [12]:
textData = df["heading_and_content"]
textData = [clean_text_round1(item) for item in textData]
textData = [clean_text_round2(item) for item in textData]
textData = [preprocess(item) for item in textData]
#remove anything that is non-ascii
textData = [remove_nonascii(item) for item in textData]
#textData = [remove_mention_hashtag(item) for item in textData]
textData = [some_words(item) for item in textData]
textData = pd.DataFrame(textData)
textData

,0
0,mumbai celebrate cool years mumbai city witnes...
1,dense lead trip power line lucknow power corpo...
2,city high level smogforming ozone chennai dang...
3,woman kid hospital inhale smoke jaipur woman k...
4,high bhogi study chennai time read bhogi poll...
...,...
17369,naval dockyard team repair oxygen plant nellor...
17370,green relook silverline implementation kerala ...
17371,cftris ozonebased disinfection combat droplet ...
17372,stay green ntpcs karimnagar plant environmenta...


In [13]:
df_needed = df[["date","heading","url"]]
df_needed

,date,heading,url
0,2010-01-01,Mumbai celebrates a cool New Year's Eve at 13....,http://timesofindia.indiatimes.com//city/mumba...
1,2010-01-02,Dense fog leads to tripping of power lines,http://timesofindia.indiatimes.com//city/luckn...
2,2010-01-09,City has high levels of smog-forming ozone,http://timesofindia.indiatimes.com//city/chenn...
3,2010-01-11,"Woman, four kids in hospital after inhaling smoke",http://timesofindia.indiatimes.com//city/jaipu...
4,2010-01-13,Pollution high on Bhogi eve: Study,http://timesofindia.indiatimes.com//city/chenn...
...,...,...,...
17369,2021-05-16,Naval Dockyard teams repair two oxygen plants ...,https://www.thehindu.com/news/national/andhra-...
17370,2021-05-24,Greens call for a relook into SilverLine imple...,https://www.thehindu.com/news/national/kerala/...
17371,2021-05-25,CFTRI’s ozone-based air disinfection system to...,https://www.thehindu.com/news/national/karnata...
17372,2021-05-28,NGT stays green nod for NTPC’s Karimnagar plant,https://www.thehindu.com/news/national/ngt-sta...


In [14]:
tokenLists = []
tweetList = []
for item in textData[0]:
    try:
        tokenLists.append(item.split(' '))
        tweetList.append(item)
    except:
        continue
len(tokenLists)

17374

In [15]:
dictionary = gensim.corpora.Dictionary(tokenLists)

#remove most frequent and less frequent words
dictionary.filter_extremes(no_below=15, no_above=0.5)
bow_corpus = [dictionary.doc2bow(doc) for doc in tokenLists]
print(len(bow_corpus))
#bow_corpus

17374


In [16]:
from time import time
from gensim.test.utils import datapath

start = time()

n_topic = 25
passes = 200

print(f"running....{n_topic}-topics - {passes}-passes")

lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=n_topic, id2word=dictionary, passes=passes,chunksize=18000)

temp_file = datapath("LDA-25-200-alldata")
lda_model.save(temp_file)
lda_model.save("LDA/LDA-25-200-alldata.gensim")

end = time()
print(f"{(end-start)/60} mins")

In [17]:
from gensim.models import CoherenceModel
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=tokenLists, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

In [18]:
vis = gensimvis.prepare(lda_model,bow_corpus,dictionary,mds='mmds', sort_topics=False)


In [ ]:
pyLDAvis.save_html(vis,'LDA/LDA-25-200-alldata.html')

In [ ]:
df = pd.DataFrame(list(lda_model[bow_corpus]))
df

In [19]:
def format_topics_sentences(ldamodel, corpus, texts):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        #each row is a list of tuple. Reverse sort on the basis of the second value of the tuple
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    dates = pd.Series(df["date"].values)
    heading = pd.Series(df["heading"].values)
    #contents = df_needed["date"]
    sent_topics_df = pd.concat([sent_topics_df, contents,dates, heading], axis=1)
    return (sent_topics_df)

In [20]:
# Format
df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=bow_corpus, texts=tweetList)
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text',"date","heading"]

# Show
df_dominant_topic.head(10)

In [21]:
from pathlib import Path
path = "LDA/files/"
Path(path).mkdir(exist_ok=True,parents=True)
df_dominant_topic.to_csv(path+"dominant_topic.csv.gz",compression="gzip")

In [24]:
pickle.dump(lda_model[bow_corpus],open("LDA/files/LDA-25-200-alldata.pickle","wb"))